# LIF animation (lava-nc)

Интерактивная демка: симуляция одиночного LIF-нейрона в lava-nc → стрим по WebSocket → отрисовка сцены (дендриты/аксон) и графиков на фронтенде.

## Поток данных (end-to-end)
- **Генерация входных спайков**: $s_{\text{in}}[t] \sim \mathrm{Bernoulli}(p=\text{rate})$ в `backend.simulate_lif`.
- **Синапс** (`Dense`, вес $w = 	ext{threshold}\cdot	ext{spike\_fraction}$): $a[t] = w\cdot s_{\text{in}}[t]$.
- **Динамика LIF** (`LIF`, плавающая точка, без мониторов):
  $$u_t = (1-\mathrm{du})\,u_{t-1} + a[t]$$
  $$v_t = (1-\mathrm{dv})\,v_{t-1} + u_t + 	ext{bias}$$
  $$s_{\text{out}}[t] = \mathbf{1}\{v_t \ge \mathrm{vth}\},\quad v_t \leftarrow 0\;\text{(reset при спайке)}$$
- **Чтение состояний**: шаг за шагом, через `lif.v.get()` и sink `RingBuffer` для `s_out`.
- **WebSocket-сервер** (`server.py`): собирает кадры \{`v`, `spike`, `input`, `t`, `threshold`\}, отправляет по `ws://<host>:8765/stream`.
- **Фронтенд** (`static/app.js`):
  - парсит кадры, обновляет буфер потенциалов/спайков;
  - рисует графики (canvas) и синхронно запускает анимацию дендритов/аксона при входном/выходном спайке.


In [ ]:
# Генерация трассы LIF офлайн (тот же код, что использует сервер)
from pathlib import Path
import json
from backend import simulate_lif

traces = simulate_lif(
    num_steps=200,
    rate=0.08,          # Bernoulli(p) для входных спайков
    threshold=1.0,
    spike_fraction=0.4, # w = threshold * spike_fraction
    dv=0.04,
    du=1.0,             # сильная утечка тока, чтобы не "залипать" в спайках
    bias=0.0,
    seed=1,
)

print("steps", len(traces["spikes"]), "spike_rate", sum(traces["spikes"])/len(traces["spikes"]))
print("first 10 v", traces["membrane_potential"][:10])
print("first 10 s", traces["spikes"][:10])

# При необходимости можно сохранить в статический JSON для отладки
Path("static/data.json").write_text(json.dumps(traces, ensure_ascii=False, indent=2), encoding="utf-8")

## Запуск бэкенда / сервера
- Зависимости (в активном `.venv`): `pip install lava-nc websockets`
- WebSocket-стрим:
```bash
python projects/04_lif_animation/server.py \
  --host 127.0.0.1 --port 8765 \
  --rate 0.08 --dv 0.04 --spike-fraction 0.4 --delay-ms 80
```
- Фронтенд (статический хостинг):
```bash
python -m http.server 8000 --directory projects/04_lif_animation/static
```
Открыть `http://localhost:8000`, фронт подключится к `ws://localhost:8765/stream` (URL задан в `static/index.html`).

## Формат кадра WebSocket
```json
{
  "v": 0.639,          // мембранный потенциал после шага
  "spike": 0,         // 1 если v>=threshold на этом шаге
  "input": 0,         // входной спайк на шаге (Bernoulli)
  "t": 9,             // глобальный такт
  "threshold": 1.0
}
```
В `app.js` при `input==1` запускается анимация дендрита, при `spike==1` \u2014 аксона; те же события попадают на графики.

In [ ]:
# Мини-утилита: подключиться к работающему серверу и посчитать частоту спайков
import asyncio, json, websockets

async def probe_ws(url="ws://localhost:8765/stream", samples=120):
    async with websockets.connect(url) as ws:
        frames = []
        for _ in range(samples):
            frames.append(json.loads(await ws.recv()))
        rate = sum(f["spike"] for f in frames)/len(frames)
        return rate, frames[:5]

rate, head = asyncio.run(probe_ws())
print("spike_rate", rate)
print("first frames", head)

## Файлы проекта
- `backend.py` \u2014 симуляция LIF (lava-nc), экспорт трасс, используется и офлайн, и сервером.
- `server.py` \u2014 WebSocket-стример: генерирует кадры `simulate_lif`, шлёт клиентам.
- `static/index.html`, `style.css`, `app.js` \u2014 фронтенд и анимации.
- `static/data.json` (опционально) \u2014 можно сгенерировать кодом выше для офлайн-отладки.